In [2]:
import numpy as np
import pandas as pd
import plotly.io as pio
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.stats import linregress

pio.renderers.default = "iframe"

In [3]:
def get_data(f=r"EURUSD_Candlestick_4_Hour_ASK_05.05.2003-16.10.2021.csv"):
    df = pd.read_csv(f)
    # df = pd.read_csv(f, nrows=500)
    df.columns=['time', 'open', 'high', 'low', 'close', 'volume']
    #Check if NA values are in data
    df=df[df['volume']!=0]
    df.reset_index(drop=True, inplace=True)
    return df
    
df = get_data()

In [4]:
def find_min_max(df, shift=5):
    df["minmax"] = 0
    
    df.loc[df['high'] == df['high'].rolling(window=shift, center=True).max(), 'minmax'] = 1  # Assign 1 to maximum points
    df.loc[df['low'] == df['low'].rolling(window=shift, center=True).min(), 'minmax'] = -1  # Assign -1 to minimum points
    return df


find_min_max(df, shift=6)

,time,open,high,low,close,volume,minmax
0,04.05.2003 21:00:00.000,1.12354,1.12354,1.12166,1.12274,95533.0976,0
1,05.05.2003 01:00:00.000,1.12242,1.12276,1.12067,1.12126,93778.5996,0
2,05.05.2003 05:00:00.000,1.12139,1.12255,1.12030,1.12113,90924.6992,0
3,05.05.2003 09:00:00.000,1.12092,1.12331,1.12049,1.12174,91254.6992,0
4,05.05.2003 13:00:00.000,1.12194,1.12900,1.12130,1.12712,308003.4083,0
...,...,...,...,...,...,...,...
28821,15.10.2021 01:00:00.000,1.15940,1.16120,1.15938,1.16082,5738.6300,0
28822,15.10.2021 05:00:00.000,1.16082,1.16189,1.15995,1.16034,13149.6600,1
28823,15.10.2021 09:00:00.000,1.16033,1.16124,1.15892,1.15923,14914.3300,0
28824,15.10.2021 13:00:00.000,1.15923,1.16103,1.15893,1.15953,20002.5300,0


In [11]:
def visualize(df):
    # https://www.youtube.com/watch?v=WVNB_6JRbl0
    # Assuming you have a DataFrame named 'df' with columns 'index', 'open', 'high', 'low', 'close', and 'minmax'

    backcandles = 20
    candleid = 4700
    
    maxim = np.array([])
    minim = np.array([])
    xxmin = np.array([])
    xxmax = np.array([])

    # min_pts = np.append(min_pts, [10, 20])
# points = np.append(points, [[10, 20, 1]], axis=0)  # Append minimum point [index, value, type]

    for i in range(candleid-backcandles, candleid+1):
        if df.iloc[i]["minmax"] == -1:  # Low
            minim = np.append(minim, df.iloc[i].low)
            xxmin = np.append(xxmin, i)  # could be i instead df.iloc[i].name
        if df.iloc[i]["minmax"] == 1:  # High
            maxim = np.append(maxim, df.iloc[i].high)
            xxmax = np.append(xxmax, i) # df.iloc[i].name

            
    def find_most_fit(minim, best_min):
        if not best_min:
            best_min = linregress(xxmin, minim)
        if not best_max:
            best_max = linregress(xxmax, maxim)
        
        for piv in xxmax:
            best_min = linregress(xxmin, minim)
        return best_min, best_max
    
    best_min, best_max = find_most_fit(minim, None, None)
    # xxmin = np.array([4685, 4692, 4696, 4700])
    # minim = np.array([1.25558, 1.25558, 1.25559, 1.25560])
    slmin, intercmin, rmin, pmin, semin = best_min
    slmax, intercmax, rmax, pmax, semax = best_max
    print(xxmin, minim)
    print(slmin, intercmin, rmin, pmin, semin)
    print(slmax, intercmax, rmax, pmax, semax)
    dfpl = df[candleid-30:candleid+30]
    
    fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                                     open=dfpl['open'],
                                     high=dfpl['high'],
                                     low=dfpl['low'],
                                     close=dfpl['close'])])
    min_points = dfpl[dfpl['minmax'] == -1]
    max_points = dfpl[dfpl['minmax'] == 1]
    fig.add_trace(go.Scatter(x=min_points.index, y=min_points["low"]-1e-3, mode="markers", marker=dict(color="red", size=5), name="Min"))
    fig.add_trace(go.Scatter(x=max_points.index, y=max_points["high"]+1e-3, mode="markers", marker=dict(color="green", size=5), name="Max"))
    
    fig.add_trace(go.Scatter(x=xxmin, y=slmin*xxmin + intercmin, mode='lines', name='min slope'))
    fig.add_trace(go.Scatter(x=xxmax, y=slmax*xxmax + intercmax, mode='lines', name='max slope'))
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.show()


visualize(df)

TypeError: visualize.<locals>.find_most_fit() takes 2 positional arguments but 3 were given